In [1]:
import sys
sys.path.append(r'C:\Users\gustavo\Documents\Data Science\08-GitHub\Portifolio/Classification/titanic_version_1')

import pandas as pd
import yaml
import pickle
import os
import json

# Carregando as configurações do arquivo YAML
yaml_path = r"C:\Users\gustavo\Documents\Data Science\08-GitHub\Portifolio\Classification\titanic_version_1\src\config.yaml"
with open(yaml_path, "r", encoding="utf-8") as f:
    config = yaml.safe_load(f)

In [2]:
params = {
    'test_data': os.path.join(config['output_data']['path'], 
                            config['output_data']['file_test']),
    'cols_2_drop' : config['feat_selection_params']['cols_2_drop'],           
    'pipe': config['pipe_feat_eng']['path'],
    'model': config['model']['path'],
    'report': config['save_reports']['path_reports'],
    'target':config['feat_selection_params']['target'],
    'submission':config['submission']['path']
    }

In [3]:
df_test = pd.read_parquet(params['test_data'])    
y_test_id = df_test[['PassengerId']].copy()
df_test.drop(
    columns=params['cols_2_drop'], 
    inplace=True)
# read pipe
pipe_path = os.path.join(
    params['model'],
    f"pipe.pkl")

with open(pipe_path, "rb") as file:
    pipe = pickle.load(file)
# appply pipe
df_test_transf = pipe.transform(df_test)    

In [4]:
model_path = os.path.join(
    params['model'],
    f"model.pkl")
# open model
with open(model_path, "rb") as file:
    model = pickle.load(file)
    
report_path = os.path.join(
    params['report'],
    "report.json")
# get cols to predict
with open(report_path, "rb") as file:
    report = json.load(file)

In [5]:
pred = model.predict(
    df_test_transf[report['Variaveis mantidas:']]
    )

In [6]:
y_test_id['Survived'] = pred

In [ ]:
pred.shape

In [7]:
y_test_id

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [ ]:
import sys
sys.path.append(r'C:\Users\gustavo\Documents\Data Science\08-GitHub\Portifolio/Classification/titanic_version_1')

import os
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier
import yaml
from mlxtend.feature_selection import SequentialFeatureSelector as SFS

# open yaml
yaml_path = r"C:\Users\gustavo\Documents\Data Science\08-GitHub\Portifolio\Classification\titanic_version_1\src\config.yaml"
with open(yaml_path, "r", encoding="utf-8") as f:
    config = yaml.safe_load(f)

In [ ]:
X_train_feat_sel = os.path.join(
            config['feat_selection']['path'],
            config['feat_selection']['X_train_file'])
    
X_val_feat_sel = os.path.join(
        config['feat_selection']['path'],
        config['feat_selection']['X_val_file'])

y_train_feat_sel = os.path.join(
        config['feat_selection']['path'],
        config['feat_selection']['y_train_file'])

y_val_feat_sel = os.path.join(
        config['feat_selection']['path'],
        config['feat_selection']['y_val_file'])
    
params = {        
    'X_train_feat_sel':X_train_feat_sel,
    'X_val_feat_sel':X_val_feat_sel,
    'y_train_feat_sel':y_train_feat_sel,
    'y_val_feat_sel':y_val_feat_sel,
    'save_plot':config['save_reports']['path_plot'],
    'save_report':config['save_reports']['path_reports']
    }

In [ ]:
X_train = pd.read_parquet(params['X_train_feat_sel'])
X_val = pd.read_parquet(params['X_val_feat_sel'])
y_train = pd.read_parquet(params['y_train_feat_sel'])
y_val = pd.read_parquet(params['y_val_feat_sel'])


sfs = SFS(
    estimator=RandomForestClassifier(random_state=23),
    k_features=X_train.shape[1]-1,  
    forward=True,  
    floating=False,  
    scoring='accuracy',
    cv=5, 
    n_jobs=-1
    )    

sfs_pipe = make_pipeline(sfs)

In [ ]:
sfs_fit = sfs_pipe.fit(X_train, y_train)

In [ ]:
results = pd.DataFrame.from_dict(sfs_pipe.named_steps['sequentialfeatureselector'].subsets_).T

In [ ]:
results

In [ ]:
metric_dict = sfs.get_metric_dict()
num_features = list(metric_dict.keys())
avg_scores = [metric_dict[k]['avg_score'] for k in num_features]
std_scores = [metric_dict[k]['std_dev'] for k in num_features]
cv_scores = [metric_dict[k]['cv_scores'] for k in num_features]  # Scores individuais de cada fold

In [ ]:
plt.figure(figsize=(12, 7))
plt.plot(
    num_features, 
    avg_scores, 
    marker='o', 
    linestyle='-', 
    color='b', 
    linewidth=2,
    markersize=8)

# Adicionar barras de erro (desvio padrão)
plt.errorbar(
    num_features, 
    avg_scores, 
    yerr=std_scores, 
    fmt='none', 
    ecolor='red', 
    elinewidth=1, 
    capsize=3, 
    alpha=0.7)

# Adicionar pontos individuais de cada fold da validação cruzada
for i, n in enumerate(num_features):
    # Espalhar os pontos horizontalmente para melhor visualização
    x_positions = np.random.normal(n, 0.05, size=len(cv_scores[i]))
    plt.scatter(x_positions, cv_scores[i], color='gray', alpha=0.6, s=30)

plt.xlabel('Número de Features', fontsize=12)
plt.ylabel('Acurácia', fontsize=12)
plt.title('Desempenho do Modelo por Número de Features Selecionadas\nCom Barras de Erro (Desvio Padrão) e Pontos Individuais de CV', fontsize=14)
plt.grid(True, linestyle='--', alpha=0.7)

In [ ]:
import pandas as pd
import yaml
import os
import pickle
import matplotlib.pyplot as plt
import dalex as dx


yaml_path = r"C:\Users\gustavo\Documents\Data Science\08-GitHub\Portifolio\Classification\titanic_version_1\src\config.yaml"
with open(yaml_path, "r", encoding="utf-8") as f:
    config = yaml.safe_load(f)

In [ ]:
X_val_feat_sel = os.path.join(
        config['feat_selection']['path'],
        config['feat_selection']['X_val_file'])   

y_val_feat_sel = os.path.join(
        config['feat_selection']['path'],
        config['feat_selection']['y_val_file'])

params_ = {
    'X_val_feat_sel': X_val_feat_sel,
    'y_val_feat_sel': y_val_feat_sel,
    'model': config['model']['path'],
    'save_plot': config['save_reports']['path_plot'],
    'removed_cols': config['save_reports']['path_reports'],
    }

In [ ]:
X_val = pd.read_parquet(params_['X_val_feat_sel'])
y_val = pd.read_parquet(params_['y_val_feat_sel'])     
    
 
model_path = os.path.join(
    params_['model'],
    f"model.pkl")    
with open(model_path, "rb") as file:
        model = pickle.load(file)

In [ ]:
explainer = dx.Explainer(model, X_val, y_val)

In [ ]:
pdp_num = explainer.model_profile(type = 'accumulated', label="ale")
pdp_num.plot(pdp_num)